# TODOs

- Understand the metrics used in the GPC paper and reimplement/copy them
- Understand the exact setup for 2D classification in the GPC paper and reimplement/copy it
- Look at which data they use and build the pipeline to use it. 

In [1]:
import numpy as np
import torch
import gpytorch
import pandas as pd
import matplotlib.pyplot as plt
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel

# Metrics

The GPC paper reports three different metrics.
- error rate: Inverse of accuracy?
- MNLL (mean? negative log likelihood): mean negative log-likelihood of a categorical using probabilities from GP
- ECE (expected calibration error): TODO

In [2]:
def get_accuracy(y_pred, y_true):
    """
    y_pred: predicted probabilities, assumes 2D tensor of shape #samples x #features
    y_true: ground truth, assumes a 1D tensor of shape #samples
    returns the accuracy of the predicted probabilities
    """
    
    max_pred = torch.argmax(y_pred, 1)
    acc = torch.mean((max_pred==y_true).float())
    return(acc)
    
def mean_neg_log_likelihood(y_pred, y_true):
    """
    y_pred: predicted probabilities, assumes 2D tensor of shape #samples x #features
    y_true: ground truth, assumes a 1D tensor of shape #samples
    returns the mean NLL of the predicted probabilities given the true labels
    """
    cat = -torch.distributions.categorical.Categorical(y_pred).log_prob(y_true)
    return(cat.mean())

def expected_calibration_error(y_pred, y_true):
    """
    y_pred: predicted probabilities, assumes 2D tensor of shape #samples x #features
    y_true: ground truth, assumes a 1D tensor of shape #samples
    returns the ECE of the predicted probabilities given the true labels
    """
    pass

def calibration_test(y_pred, y_true, nbins=10):
    '''
    COPIED FROM DGP PAPER
    y_pred: predicted probabilities, assumes 2D tensor of shape #samples x #features
    y_true: ground truth, assumes a 1D tensor of shape #samples
    Returns ece:  Expected Calibration Error
            conf: confindence levels (as many as nbins)
            accu: accuracy for a certain confidence level
                  We are interested in the plot confidence vs accuracy
            bin_sizes: how many points lie within a certain confidence level
    '''
    edges = torch.linspace(0, 1, nbins+1)
    accu = torch.zeros(nbins)
    conf = torch.zeros(nbins)
    bin_sizes = torch.zeros(nbins)
    # Multiclass problems are treated by considering the max
    pred = torch.argmax(y_pred, dim=1)
    prob = torch.max(y_pred, dim=1)[0]
    
    #
    y_true = y_true.view(-1)
    prob = prob.view(-1)
    for i in range(nbins):
        idx_in_bin = (prob > edges[i]) & (prob <= edges[i+1])
        bin_sizes[i] = max(sum(idx_in_bin), 1)
        accu[i] = torch.sum(y_true[idx_in_bin] == pred[idx_in_bin]) / bin_sizes[i]
        conf[i] = (edges[i+1] + edges[i]) / 2
    ece = torch.sum(torch.abs(accu - conf) * bin_sizes) / torch.sum(bin_sizes)
    return ece, conf, accu, bin_sizes
    

# Setup for GPC & LB(Beta)+GP

create function that makes train-test split

create function that gets the right number of inducing points from test data

create functions that take in data and return a fitted GP

create a function that takes in the fitted GP and returns all the relevant measurements

In [3]:
### load an example dataset
data_files = "/home/marius/Desktop/Laplace_Matching_for_GLMs/Beta_Experiments/data/"

filename = "EEG_Eye_State.csv"
EEG_df = pd.read_csv(data_files + filename, header=None)
EEG_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0


In [4]:
# make test-train split
from sklearn.model_selection import train_test_split

X_EEG = EEG_df.values[:,:-1]
Y_EEG = EEG_df.values[:,-1]

# The original paper chose 10980 training points and 4000 test points
test_size_EEG = 4000/(10980+4000)
X_EEG_train, X_EEG_test, y_EEG_train, y_EEG_test = train_test_split(X_EEG, Y_EEG, test_size=test_size_EEG, random_state=42)
print(np.shape(X_EEG_train))
print(np.shape(X_EEG_test))
print(np.shape(y_EEG_train))
print(np.shape(y_EEG_test))
X_EEG_train, X_EEG_test, y_EEG_train, y_EEG_test = torch.tensor(X_EEG_train).float(), torch.tensor(X_EEG_test).float(), \
                                                   torch.tensor(y_EEG_train).long(), torch.tensor(y_EEG_test).long()
print(y_EEG_test[:20])

(10980, 14)
(4000, 14)
(10980,)
(4000,)
tensor([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0])


In [ ]:
# create k-means cluster for the inducing points

def k_means_inducing_points(train_x, train_y, num_inducing_points):
    
    pass

In [5]:
# ignore for now and just use random subsets

num_inducing_points = 1000
inducing_idxs = np.random.choice(10980, num_inducing_points, replace=False)
X_EEG_train_induced = torch.tensor(X_EEG_train[inducing_idxs]).float()
y_EEG_train_induced = torch.tensor(y_EEG_train[inducing_idxs]).long()

# TODO use k-means clustering and conjugacy to create better inducing points

/tmp/ipykernel_9298/269273252.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_EEG_train_induced = torch.tensor(X_EEG_train[inducing_idxs]).float()
/tmp/ipykernel_9298/269273252.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_EEG_train_induced = torch.tensor(y_EEG_train[inducing_idxs]).long()


In [6]:
# create a generic function for the GPC
# returns dirichlet gaussian process classification model and likelihood

def create_DGP_model(train_x, train_y, learn_additional_noise=True):

    class DirichletGPModel(ExactGP):
        def __init__(self, train_x, train_y, likelihood, num_classes):
            super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = ConstantMean(batch_shape=torch.Size((num_classes,)))
            self.covar_module = ScaleKernel(
                RBFKernel(batch_shape=torch.Size((num_classes,))),
                batch_shape=torch.Size((num_classes,)),
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    # initialize likelihood and model
    # we let the DirichletClassificationLikelihood compute the targets for us
    likelihood = DirichletClassificationLikelihood(train_y, learn_additional_noise=learn_additional_noise)
    model = DirichletGPModel(train_x, likelihood.transformed_targets, likelihood, num_classes=likelihood.num_classes)
    return(model, likelihood)

DGP_model, DGP_likelihood = create_DGP_model(X_EEG_train_induced, y_EEG_train_induced)
print(DGP_model)

DirichletGPModel(
  (likelihood): DirichletClassificationLikelihood(
    (noise_covar): FixedGaussianNoise()
    (second_noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)


In [7]:
def train_DGP_model(train_x, model, likelihood, num_iter=500, lr=0.1, report_iter=50):
    
    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(num_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, likelihood.transformed_targets).sum()
        loss.backward()
        #print(model.likelihood)
        if i % report_iter == 0:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, num_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.mean().item(),
                model.likelihood.noise_covar.noise.mean().item()
            ))
        optimizer.step()
        
    return(model, likelihood)

DGP_model, DGP_likelihood = train_DGP_model(X_EEG_train_induced,
                                            DGP_model, DGP_likelihood, num_iter=500, lr=0.1, report_iter=50)

Iter 1/500 - Loss: 7.102   lengthscale: 0.693   noise: 2.652
Iter 51/500 - Loss: 5.182   lengthscale: 0.693   noise: 2.652
Iter 101/500 - Loss: 5.170   lengthscale: 0.693   noise: 2.652
Iter 151/500 - Loss: 5.172   lengthscale: 0.693   noise: 2.652
Iter 201/500 - Loss: 5.174   lengthscale: 0.693   noise: 2.652
Iter 251/500 - Loss: 5.173   lengthscale: 0.693   noise: 2.652
Iter 301/500 - Loss: 5.172   lengthscale: 0.693   noise: 2.652
Iter 351/500 - Loss: 5.174   lengthscale: 0.693   noise: 2.652
Iter 401/500 - Loss: 5.171   lengthscale: 0.693   noise: 2.652
Iter 451/500 - Loss: 5.169   lengthscale: 0.693   noise: 2.652


In [18]:
# create a generic function for the LM+Beta process

def LB_beta(alpha, beta):
    
    mu = np.log(alpha/beta)
    var = (alpha+beta)/(alpha*beta)
    return(mu, var)

def transform_y_beta_LM(train_y, eps_alpha=0.1, eps_beta=0.1):

    train_alphas = torch.ones_like(train_y) * eps_alpha
    train_alphas[train_y > 0.5] += 1
    train_betas = torch.ones_like(train_y) * eps_beta
    train_betas[train_y < 0.5] += 1

    train_mu_LB, train_var_LB = LB_beta(train_alphas, train_betas)
    return(train_mu_LB, train_var_LB)

def create_LMGP_model(train_x, train_y_mu, train_y_var, learn_additional_noise=False):
    
    # We will use the simplest form of GP model, exact inference
    class LMGPModel(ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(LMGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    # initialize likelihood and model
    likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=torch.sqrt(train_y_var), 
                                                                   learn_additional_noise=learn_additional_noise)
    model = LMGPModel(train_x, train_y_mu, likelihood)
    return(model, likelihood)

y_EEG_train_induced_mu, y_EEG_train_induced_var = transform_y_beta_LM(y_EEG_train_induced)
LMGP_model, LMGP_likelihood = create_LMGP_model(X_EEG_train_induced, y_EEG_train_induced_mu,
                                                y_EEG_train_induced_var, learn_additional_noise=False)
print(LMGP_model)

LMGPModel(
  (likelihood): FixedNoiseGaussianLikelihood(
    (noise_covar): FixedGaussianNoise()
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)


In [16]:
# generic function to train LM+Beta GP
def train_LMGP_model(train_x, train_y_mu, model, likelihood, num_iter=500, lr=0.1, report_iter=50):
    
    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(num_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y_mu)
        loss.backward()
        #print(model.likelihood)
        if i % report_iter == 0:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f' % (
                i + 1, num_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.mean().item()
            ))
        optimizer.step()
        
    return(model, likelihood)

LMGP_model, LMGP_likelihood = train_LMGP_model(X_EEG_train_induced, y_EEG_train_induced_mu,
                                            LMGP_model, LMGP_likelihood, num_iter=500, lr=0.1, report_iter=50)

Iter 1/500 - Loss: 2.305   lengthscale: 0.693
Iter 51/500 - Loss: 2.281   lengthscale: 0.693
Iter 101/500 - Loss: 2.281   lengthscale: 0.693
Iter 151/500 - Loss: 2.281   lengthscale: 0.693
Iter 201/500 - Loss: 2.281   lengthscale: 0.693
Iter 251/500 - Loss: 2.281   lengthscale: 0.693
Iter 301/500 - Loss: 2.281   lengthscale: 0.693
Iter 351/500 - Loss: 2.281   lengthscale: 0.693
Iter 401/500 - Loss: 2.281   lengthscale: 0.693
Iter 451/500 - Loss: 2.281   lengthscale: 0.693


In [12]:
# Evaluate DGP
def evaluate_DGP(model, likelihood, test_x, test_y, num_samples=1000):
    
    model.eval()
    likelihood.eval()

    with gpytorch.settings.fast_pred_var(), torch.no_grad():
        test_dist = model(test_x)

    pred_samples = test_dist.sample(torch.Size((num_samples,)))
    pred_samples_exp = pred_samples.exp()
    #probs
    pred_samples_norm = (pred_samples_exp / pred_samples_exp.sum(-2, keepdim=True))
    pred_probs = pred_samples_norm.mean(0)
    pred_probs = torch.swapaxes(pred_probs, 0, 1)
    
    print(pred_probs.size(), test_y.size())
    #print(pred_probs[:50], test_y[:50])
    
    # evaluate
    acc = get_accuracy(pred_probs, test_y)
    mnll = mean_neg_log_likelihood(pred_probs, test_y)
    ece, conf, accu, bin_sizes = calibration_test(pred_probs, test_y, nbins=10)
    
    return(acc, mnll, ece)
    
evaluate_DGP(DGP_model, DGP_likelihood, X_EEG_test, y_EEG_test)

torch.Size([4000, 2]) torch.Size([4000])


(tensor(0.5305), tensor(0.8109), tensor(0.2201))

In [17]:
# Evaluate LG+Beta GP
def logistic(x):
    return(1 / (1 + np.exp(-x)))

def evaluate_LMGP(model, likelihood, test_x, test_y, num_samples=1000):
    
    model.eval()
    likelihood.eval()

    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(test_x))

    pred_mean = observed_pred.mean
    pred_probs = logistic(pred_mean)
    # make 2D
    pred_probs = torch.cat([1-pred_probs.view(-1,1), pred_probs.view(-1,1)], dim=1)
    print(pred_probs.size())
    
    print(pred_probs.size(), test_y.size())
    #print(pred_probs[:50], test_y[:50])
    
    # evaluate
    acc = get_accuracy(pred_probs, test_y)
    mnll = mean_neg_log_likelihood(pred_probs, test_y)
    ece, conf, accu, bin_sizes = calibration_test(pred_probs, test_y, nbins=10)
    
    return(acc, mnll, ece)
    
evaluate_LMGP(LMGP_model, LMGP_likelihood, X_EEG_test, y_EEG_test)

torch.Size([4000, 2])
torch.Size([4000, 2]) torch.Size([4000])


(tensor(0.5305), tensor(0.6994), tensor(0.0206))

# find all datasets and run all measurements